In [1]:
import os
import sys
sys.path.append(os.pardir)
import numpy as np
from numpy.random import *
from scipy import ceil, complex64, float64, hamming, zeros
from matplotlib import pylab as plt
import xml.etree.ElementTree as et
import argparse
import time
from cmf.cnimf import CNIMF
from matplotlib import pylab as plt
%matplotlib inline

In [2]:
convolution_width = 3
true_n_components = 6

In [3]:
parser = argparse.ArgumentParser(description='structure detection test')
parser.add_argument('-s', '--seed_number', \
                    action='store', \
                    nargs='?', \
                    const=None, \
                    default=0, \
                    type=int, \
                    choices=None, \
                    help='seed_number', \
                    metavar=None)
parser.add_argument('-d', '--dat_dir', \
                    action='store', \
                    nargs='?', \
                    const=None, \
                    default='../dat/structure_detection', \
                    type=str, \
                    choices=None, \
                    help='Directory where npy files will be stored.', \
                    metavar=None)
parser.add_argument('-n', '--dat_npz', \
                    action='store', \
                    nargs='?', \
                    const=None, \
                    default='structure_result_{}_{}_{}.npz'.format(convolution_width, true_n_components, time.strftime('%Y%m%d%H%M')), \
                    type=str, \
                    choices=None, \
                    help='npz file name', \
                    metavar=None)


_StoreAction(option_strings=['-n', '--dat_npz'], dest='dat_npz', nargs='?', const=None, default='structure_result_3_6_201705131528.npz', type=<class 'str'>, choices=None, help='npz file name', metavar=None)

In [4]:
args = parser.parse_args([])

In [5]:
seed(args.seed_number)


In [6]:
n_tests = 50
n_criteria = 5
n_samples_list = [100, 200, 300, 400, 500, 600, 700, 800]
missing_rate_list = [0.2, 0.6]
data_dim = 12
convolution_width = convolution_width
true_n_components_list = [true_n_components]
gamma_shape = 2.0
gamma_rate = 2.0
base_max = 10.0
best_n_bases = np.zeros([len(true_n_components_list), len(n_samples_list), n_tests, n_criteria])
completion_error = np.zeros([len(true_n_components_list), len(missing_rate_list), len(n_samples_list), n_tests, n_criteria])
accuracy = np.zeros([len(true_n_components_list), len(n_samples_list), n_criteria])

In [7]:
# %pdb
# import warnings
# warnings.filterwarnings('error')
for i_n_components in range(len(true_n_components_list)):
    true_n_components = true_n_components_list[i_n_components]
    for i_n_samples in range(len(n_samples_list)):
        n_samples = n_samples_list[i_n_samples]
        print('n_samples', n_samples)
        for i_test in range(n_tests):
            print('i_test', i_test)
            true_activation = np.random.gamma(gamma_shape, 1.0 / gamma_rate, [n_samples, true_n_components])
            true_base = np.random.uniform(0.0, base_max, [convolution_width, true_n_components, data_dim])
            X = np.random.poisson(CNIMF.convolute(true_activation, true_base))
            arg_dict = dict(
                convolution_max = convolution_width,
                component_max = X.shape[1],
                true_width = convolution_width,
                gamma_shape = gamma_shape,
                gamma_rate = gamma_rate,
                base_max = base_max,
                convergence_threshold = 0.00001,
                loop_max = 1000)
            factorizer = CNIMF(**arg_dict)
            filtre = np.ones(X.shape)
            factorizer.fit(X, None, filtre)
            for i_criterion in range(n_criteria):
                best_n_bases[i_n_components, i_n_samples, i_test, i_criterion] = factorizer.criteria[i_criterion].best_structure[1]
            for i_missing_rate in range(len(missing_rate_list)):
                missing_rate = missing_rate_list[i_missing_rate]
                print(missing_rate)
                factorizer = CNIMF(**arg_dict)
                filtre = np.random.binomial(1, missing_rate, X.shape)
                factorizer.fit(X, None, filtre)
                for i_criterion in range(n_criteria):
                    completion_error[i_n_components, i_missing_rate, i_n_samples, i_test, i_criterion] = factorizer.criteria[i_criterion].completion_error/np.prod(X.shape)
                print(completion_error[i_n_components, i_missing_rate, i_n_samples, i_test, :])
    accuracy[i_n_components, :, :] = np.mean(best_n_bases[i_n_components, :, :, :]==true_n_components, axis = 1)

n_samples 100
i_test 0
0.2
[ 2.21760985  2.15829958  2.15829958  2.15829958  2.15829958]
0.6
[ 0.78353297  0.91237202  0.91237202  0.91237202  0.74610762]
i_test 1
0.2
[ 2.09884928  1.92804682  1.92804682  1.92804682  1.92804682]
0.6
[ 0.81715595  0.81925023  0.81925023  0.81925023  0.77784577]
i_test 2
0.2
[ 2.6944651   1.96885931  2.6944651   2.6944651   1.96885931]
0.6
[ 0.96982016  0.93872259  1.12181357  0.93872259  0.94801978]
i_test 3
0.2
[ 2.22654759  1.78510978  1.78510978  1.78510978  1.78510978]
0.6
[ 0.85879402  0.8913543   0.98718342  0.98718342  0.84979858]
i_test 4
0.2
[ 2.77418451  2.71910328  2.71910328  2.71910328  2.71910328]
0.6
[ 1.06163229  0.89476573  1.17970384  0.89476573  0.88947125]
i_test 5
0.2
[ 3.17470514  2.57190416  2.57190416  2.57190416  2.57190416]
0.6
[ 1.03024372  0.8641387   1.03024372  0.8641387   0.86937545]
i_test 6
0.2
[ 2.66132179  2.92600285  2.92600285  2.92600285  2.92600285]
0.6
[ 0.9366711   0.90865796  0.90865796  0.90865796  0.85906445]

In [8]:
np.savez(args.dat_dir + '/' + args.dat_npz,
         accuracy=accuracy,
         completion_error=completion_error,
         best_n_bases=best_n_bases,
         n_samples_list = n_samples_list,
         missing_rate_list = missing_rate_list, 
         true_n_components_list = true_n_components_list,
         convolution_width = convolution_width)

In [9]:
# npz = np.load(args.dat_dir + '/artificial_result_201701210117.npz')

In [10]:
accuracy

array([[[ 0.  ,  0.02,  0.52,  0.12,  0.  ],
        [ 0.  ,  0.22,  0.  ,  0.4 ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.3 ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.4 ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ]]])

In [11]:
completion_error

array([[[[[ 2.21760985,  2.15829958,  2.15829958,  2.15829958,  2.15829958],
          [ 2.09884928,  1.92804682,  1.92804682,  1.92804682,  1.92804682],
          [ 2.6944651 ,  1.96885931,  2.6944651 ,  2.6944651 ,  1.96885931],
          ..., 
          [ 2.01719646,  1.60120208,  1.60120208,  1.60120208,  1.60120208],
          [ 2.83291896,  2.36947701,  2.83291896,  2.36947701,  2.36947701],
          [ 2.18711685,  2.51335643,  2.51335643,  2.51335643,  2.51335643]],

         [[ 2.46935824,  2.46935824,  2.46935824,  1.98912245,  1.98912245],
          [ 2.14658312,  2.14658312,  2.14658312,  1.50986904,  1.50986904],
          [ 1.91804503,  2.13267548,  2.13267548,  1.51107545,  1.51107545],
          ..., 
          [ 2.46846216,  2.46846216,  2.46846216,  2.46846216,  1.88186801],
          [ 1.95323681,  1.95323681,  1.95323681,  1.64522809,  1.64522809],
          [ 1.90160204,  2.52458194,  2.52458194,  1.98593795,  1.98593795]],

         [[ 1.75312516,  2.17780512,  2.